In [1]:
from mediapipe.python.solutions import pose as mp_pose
from pathlib import Path
import pandas as pd
from tqdm.auto import tqdm
import cv2
import numpy as np

2023-08-13 13:48:05.034732: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-13 13:48:07.449265: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
BASE_PATH = Path('/media/eros/BackupMae/datasets/Minds')
OUT_PATH = Path('./Outs')
meta_df = pd.read_csv(OUT_PATH / 'dataset_metadata.csv.gz', 
                      dtype={
                          'filename': 'string',
                          'stem': 'string',
                          'pose_id': 'int64',
                          'word': 'category',
                          'sinalizador': 'int64',
                          'repetition': 'int64',
                      }, index_col=0)
meta_df.head()

,filename,stem,pose_id,word,sinalizador,repetition
0,/media/eros/BackupMae/datasets/Minds/Sinalizad...,01AcontecerSinalizador07-1,1,Acontecer,7,1
1,/media/eros/BackupMae/datasets/Minds/Sinalizad...,01AcontecerSinalizador07-2,1,Acontecer,7,2
2,/media/eros/BackupMae/datasets/Minds/Sinalizad...,01AcontecerSinalizador07-3,1,Acontecer,7,3
3,/media/eros/BackupMae/datasets/Minds/Sinalizad...,01AcontecerSinalizador07-4,1,Acontecer,7,4
4,/media/eros/BackupMae/datasets/Minds/Sinalizad...,01AcontecerSinalizador07-5,1,Acontecer,7,5


In [3]:
test_video = cv2.VideoCapture(meta_df.loc[0, 'filename'])
ret, frame = test_video.read()
ret

True

In [4]:
with mp_pose.Pose() as pose_tracker:
    result = pose_tracker.process(image=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    pose_landmarks = result.pose_landmarks
len(pose_landmarks.landmark)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


33

In [5]:
list(meta_df.head().itertuples(index=True))

[Pandas(Index=0, filename='/media/eros/BackupMae/datasets/Minds/Sinalizador07/Canon/01AcontecerSinalizador07-1.mp4', stem='01AcontecerSinalizador07-1', pose_id=1, word='Acontecer', sinalizador=7, repetition=1),
 Pandas(Index=1, filename='/media/eros/BackupMae/datasets/Minds/Sinalizador07/Canon/01AcontecerSinalizador07-2.mp4', stem='01AcontecerSinalizador07-2', pose_id=1, word='Acontecer', sinalizador=7, repetition=2),
 Pandas(Index=2, filename='/media/eros/BackupMae/datasets/Minds/Sinalizador07/Canon/01AcontecerSinalizador07-3.mp4', stem='01AcontecerSinalizador07-3', pose_id=1, word='Acontecer', sinalizador=7, repetition=3),
 Pandas(Index=3, filename='/media/eros/BackupMae/datasets/Minds/Sinalizador07/Canon/01AcontecerSinalizador07-4.mp4', stem='01AcontecerSinalizador07-4', pose_id=1, word='Acontecer', sinalizador=7, repetition=4),
 Pandas(Index=4, filename='/media/eros/BackupMae/datasets/Minds/Sinalizador07/Canon/01AcontecerSinalizador07-5.mp4', stem='01AcontecerSinalizador07-5', pose

In [6]:
videos_landmarks = []
append_video_landmarks = videos_landmarks.append

for video in tqdm(list(meta_df.itertuples(index=True))):
    base_row = [video.Index]
    cap = cv2.VideoCapture(video.filename)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    tq = tqdm(total=frame_count)
    with mp_pose.Pose() as pose_tracker:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                # tq.write("Can't receive frame (stream end?). Exiting ...")
                break
            result = pose_tracker.process(image=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            pose_landmarks = result.pose_landmarks
            if pose_landmarks is not None:
                landmarks = pose_landmarks.landmark
                assert len(landmarks) == 33, f'Unexpected number of predicted pose landmarks: {len(landmarks)}'
                list_landmarks = [[lmk.x, lmk.y, lmk.z] for lmk in landmarks]
                append_video_landmarks(base_row + list_landmarks)
            tq.update(1)
len(videos_landmarks)

  0%|          | 0/1158 [00:00<?, ?it/s]

  0%|          | 0/142 [00:00<?, ?it/s]

  0%|          | 0/121 [00:00<?, ?it/s]

  0%|          | 0/136 [00:00<?, ?it/s]

  0%|          | 0/127 [00:00<?, ?it/s]

...

  0%|          | 0/143 [00:00<?, ?it/s]

  0%|          | 0/174 [00:00<?, ?it/s]

  0%|          | 0/132 [00:00<?, ?it/s]

  0%|          | 0/167 [00:00<?, ?it/s]

161439

In [7]:
video_id = np.array([x[0] for x in videos_landmarks])
final_landmarks = np.array([x[1:] for x in videos_landmarks])
final_landmarks.shape 

(161439, 33, 3)

In [8]:
np.save(str(OUT_PATH / 'video_id'), video_id)
np.save(str(OUT_PATH / 'landmarks'), final_landmarks)